In [48]:
# Import required libraries for left hand visualization
import jax
import jax.numpy as jnp
from IPython.display import HTML

# Import directly from brax for rendering
from brax.v1.io import html

# Import the left hand environment directly from our repository
from qdax.environments.left_hand import LeftHand

print("✅ All libraries imported successfully!")
print("✅ Left hand environment ready to use!")

✅ All libraries imported successfully!
✅ Left hand environment ready to use!


In [49]:
# Initialize the Left Hand environment
env = LeftHand()

# Create a random key for initialization
key = jax.random.PRNGKey(42)

# Reset the environment to get initial state
reset_fn = jax.jit(env.reset)
state = reset_fn(key)

print("✅ Left Hand environment initialized!")

# Use the new jax.tree.map instead of deprecated jax.tree_map
try:
    # Try the new function first (JAX v0.4.25+)
    print(f"State shape: {jax.tree.map(lambda x: x.shape, state)}")
except AttributeError:
    # Fallback to the older function for compatibility
    print(f"State shape: {jax.tree_util.tree_map(lambda x: x.shape, state)}")

# Print environment information without accessing .dt attribute
print(f"Action size: {env.action_size}")
print(f"Observation size: {env.observation_size}")
print("✅ Environment ready for rendering!")

✅ Found XML file at: C:\Users\eduar\Documents\GitHub\OI_QDax\qdax\environments\..\..\assets\left_hand.xml


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\eduar\\Documents\\GitHub\\OI_QDax\\qdax\\environments\\..\\..\\assets\\assets\\palm_left.obj'

In [ ]:
# Create render of the Left Hand
# Generate a simple trajectory by taking a few steps with random actions
states = [state]
current_state = state

# Take more steps with larger movements to see the hand in motion
for i in range(20):
    # Generate random action with larger magnitude for visibility
    action = jax.random.normal(key, shape=(env.action_size,)) * 0.3
    key, subkey = jax.random.split(key)
    
    # Step the environment
    current_state = env.step(current_state, action)
    states.append(current_state)

print(f"✅ Generated trajectory with {len(states)} states")

# Extract QP (position and rotation) from states for rendering
qps = []
for state in states:
    if hasattr(state, 'qp'):
        qps.append(state.qp)
    elif hasattr(state, 'pipeline_state'):
        qps.append(state.pipeline_state.qp)
    else:
        # Try to extract qp from the state structure
        try:
            qps.append(state.pipeline_state.qp)
        except:
            print(f"⚠️ Could not extract QP from state, state attributes: {dir(state)}")
            # Create a simple visualization with the first state only
            qps = [states[0].pipeline_state.qp]
            break

print(f"✅ Extracted {len(qps)} QP states for rendering")

# Debug: Print some information about the QP
if qps:
    first_qp = qps[0]
    print(f"QP pos shape: {first_qp.pos.shape}")
    print(f"QP rot shape: {first_qp.rot.shape}")
    print(f"Pos range: min={jnp.min(first_qp.pos):.3f}, max={jnp.max(first_qp.pos):.3f}")

# Create HTML visualization with larger height for better visibility
html_string = html.render(env.sys, qps, height=600)

# Display the render
HTML(html_string)

In [50]:
# Debug: Inspect the system to understand what we're rendering
print("🔍 System Information:")
print(f"System type: {type(env.sys)}")

# Check system attributes
print(f"System attributes: {[attr for attr in dir(env.sys) if not attr.startswith('_')]}")

# Check body information
if hasattr(env.sys, 'body'):
    body = env.sys.body
    print(f"Body type: {type(body)}")
    print(f"Body attributes: {[attr for attr in dir(body) if not attr.startswith('_')]}")
    
    # Try to get body information
    if hasattr(body, 'name'):
        print(f"Body name: {body.name}")
    if hasattr(body, 'mass'):
        print(f"Body mass shape: {body.mass.shape if hasattr(body.mass, 'shape') else 'No shape'}")

# Check joint information
if hasattr(env.sys, 'joint'):
    joint = env.sys.joint
    print(f"Joint type: {type(joint)}")
    print(f"Joint attributes: {[attr for attr in dir(joint) if not attr.startswith('_')]}")

# Check config information
if hasattr(env.sys, 'config'):
    config = env.sys.config
    print(f"Config type: {type(config)}")
    print(f"Config attributes: {[attr for attr in dir(config) if not attr.startswith('_')]}")

# Check degrees of freedom
if hasattr(env.sys, 'dof'):
    print(f"Degrees of freedom: {env.sys.dof}")
    
# Check if we have actuators
if hasattr(env.sys, 'actuator'):
    actuator = env.sys.actuator
    print(f"Actuator type: {type(actuator)}")

print("\n🎨 Creating static visualization...")
try:
    # Just show the initial state
    static_html = html.render(env.sys, [qps[0]], height=600)
    print("✅ Static visualization created")
    HTML(static_html)
except Exception as e:
    print(f"❌ Error creating static visualization: {e}")
    
    # Print more debug information
    print(f"QP type: {type(qps[0])}")
    print(f"QP attributes: {[attr for attr in dir(qps[0]) if not attr.startswith('_')]}")
    
    # Try alternative approach
    print("🔄 Trying simpler rendering...")
    try:
        # Create the simplest possible render
        simple_html = html.render(env.sys, qps[:1])
        HTML(simple_html)
    except Exception as e2:
        print(f"❌ Alternative rendering failed: {e2}")
        print("The system might not be properly configured for rendering")

🔍 System Information:
System type: <class 'brax.v1.physics.system.System'>
System attributes: ['actuators', 'body', 'colliders', 'config', 'default_angle', 'default_qp', 'forces', 'info', 'integrator', 'joints', 'num_actuators', 'num_bodies', 'num_forces_dof', 'num_joint_dof', 'num_joints', 'step', 'zero_info']
Body type: <class 'brax.v1.physics.bodies.Body'>
Body attributes: ['active', 'idx', 'impulse', 'index', 'inertia', 'mass']
Body mass shape: (5,)
Config type: <class 'brax.physics.config_pb2.Config'>
Config attributes: ['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'NamePair', 'ParseFromString', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', 'actuators', 'angular_damping', 'baumgarte_erp', 'bodies', 'collide_include', 'collider_cutoff', 'defaults

In [31]:
# Verify leap hand model configuration and assets
import os
from pathlib import Path

print("🔍 Checking leap hand model configuration...")

# Check if assets directory exists
assets_dir = Path("assets")
if assets_dir.exists():
    print("✅ Assets directory found")
    assets_files = list(assets_dir.glob("*"))
    print(f"Assets files: {[f.name for f in assets_files]}")
    
    # Check for specific hand files
    left_hand_xml = assets_dir / "left_hand.xml"
    scene_left_xml = assets_dir / "scene_left.xml"
    
    if left_hand_xml.exists():
        print(f"✅ Found left_hand.xml")
        # Read and check content
        with open(left_hand_xml, 'r') as f:
            content = f.read()
            print(f"XML file size: {len(content)} characters")
            # Check for important elements
            if '<mujoco>' in content:
                print("✅ MuJoCo XML structure found")
            if '<body' in content:
                print("✅ Body elements found")
            if '<joint' in content:
                print("✅ Joint elements found")
            if '<geom' in content:
                print("✅ Geom elements found")
            if '<actuator' in content:
                print("✅ Actuator elements found")
            else:
                print("⚠️ No actuators found - this might be the issue!")
    else:
        print("❌ left_hand.xml not found")
    
    if scene_left_xml.exists():
        print("✅ Found scene_left.xml")
    else:
        print("❌ scene_left.xml not found")
        
else:
    print("❌ Assets directory not found")
    print("Current directory contents:")
    current_dir = Path(".")
    print([f.name for f in current_dir.iterdir() if f.is_file() or f.is_dir()])

# Check the environment's XML configuration
print("\n🔍 Checking environment XML configuration...")
if hasattr(env, '_xml_path') or hasattr(env, 'xml_path'):
    xml_path = getattr(env, '_xml_path', None) or getattr(env, 'xml_path', None)
    print(f"Environment XML path: {xml_path}")
    
    if xml_path and os.path.exists(xml_path):
        print("✅ XML file exists")
        with open(xml_path, 'r') as f:
            xml_content = f.read()
            print(f"XML content preview (first 500 chars):")
            print(xml_content[:500])
    else:
        print("❌ XML file not found at specified path")

# Check if the system has the required elements
print("\n🔍 Checking system completeness...")
if hasattr(env.sys, 'body') and hasattr(env.sys.body, 'name'):
    body_names = env.sys.body.name
    if hasattr(body_names, 'shape'):
        print(f"Number of bodies: {body_names.shape[0]}")
        print(f"First few body names: {body_names[:min(5, len(body_names))]}")
    else:
        print(f"Body name: {body_names}")

if hasattr(env.sys, 'joint') and hasattr(env.sys.joint, 'name'):
    joint_names = env.sys.joint.name
    if hasattr(joint_names, 'shape'):
        print(f"Number of joints: {joint_names.shape[0]}")
        print(f"First few joint names: {joint_names[:min(5, len(joint_names))]}")
    else:
        print(f"Joint name: {joint_names}")

print("\n💡 Potential issues:")
print("1. Missing actuators in XML (render will be gray)")
print("2. Incorrect mesh files or paths")
print("3. Missing collision/visual geometries")
print("4. Incorrect MuJoCo XML structure")
print("5. Missing material definitions")

🔍 Checking leap hand model configuration...
✅ Assets directory found
Assets files: ['left_hand.xml', 'right_hand.xml', 'scene_left.xml', 'scene_right.xml']
✅ Found left_hand.xml
XML file size: 23409 characters
✅ Body elements found
✅ Joint elements found
✅ Geom elements found
✅ Actuator elements found
✅ Found scene_left.xml

🔍 Checking environment XML configuration...

🔍 Checking system completeness...

💡 Potential issues:
1. Missing actuators in XML (render will be gray)
2. Incorrect mesh files or paths
3. Missing collision/visual geometries
4. Incorrect MuJoCo XML structure
5. Missing material definitions


In [32]:
# Alternative visualization approach with better camera setup
print("🎨 Attempting alternative visualization...")

# Try to create a more visible visualization
try:
    # Get the QP from the first state for debugging
    qp = qps[0]
    
    # Print detailed QP information
    print(f"QP position shape: {qp.pos.shape}")
    print(f"QP rotation shape: {qp.rot.shape}")
    print(f"Position values range: {jnp.min(qp.pos):.4f} to {jnp.max(qp.pos):.4f}")
    print(f"Position mean: {jnp.mean(qp.pos):.4f}")
    
    # Check if positions are non-zero
    if jnp.allclose(qp.pos, 0.0):
        print("⚠️ All positions are zero - model might not be loaded correctly")
    
    # Try with custom rendering parameters
    try:
        # Simple single-frame render
        html_vis = html.render(env.sys, [qp], height=800)
        print("✅ Single frame visualization created")
        HTML(html_vis)
    except Exception as e:
        print(f"❌ Single frame render failed: {e}")
        
        # Try with a different approach - create a simple trajectory
        print("🔄 Trying with simple trajectory...")
        try:
            # Create a minimal trajectory with just 2 states
            simple_qps = qps[:2]
            simple_html = html.render(env.sys, simple_qps, height=800)
            HTML(simple_html)
        except Exception as e2:
            print(f"❌ Simple trajectory failed: {e2}")
            
            # Last resort - check if we can at least see the system info
            print("🔍 Final debug information:")
            print(f"System DOF: {env.sys.dof if hasattr(env.sys, 'dof') else 'Unknown'}")
            
            # Try to access system configuration
            if hasattr(env.sys, 'config'):
                config = env.sys.config
                print(f"System config type: {type(config)}")
                if hasattr(config, 'dt'):
                    print(f"System dt: {config.dt}")
                if hasattr(config, 'substeps'):
                    print(f"System substeps: {config.substeps}")
                    
            # Check for mesh files
            if hasattr(env.sys, 'body'):
                body = env.sys.body
                if hasattr(body, 'name'):
                    print(f"Body names available: {body.name}")
            
            print("❌ All visualization attempts failed")
            print("💡 The model might be missing visual meshes or materials")
            print("💡 Try checking the assets directory for .obj/.stl files")
            print("💡 The XML might need proper visual/collision geoms defined")
            
except Exception as e:
    print(f"❌ Critical error in visualization: {e}")
    print("The environment might not be properly configured for rendering")

🎨 Attempting alternative visualization...
QP position shape: (5, 3)
QP rotation shape: (5, 4)
Position values range: -0.1013 to 0.0221
Position mean: -0.0332
✅ Single frame visualization created


In [33]:
# Inspect and fix the LeftHand model loading
from brax.io import mjcf
from etils import epath
import os
from pathlib import Path

print("🔍 Inspecting LeftHand model loading...")

# Check how LeftHand loads its model
print(f"LeftHand class: {LeftHand}")
print(f"LeftHand MRO: {LeftHand.__mro__}")

# Let's look at the LeftHand source or try to understand its initialization
try:
    # Try to get the XML path used by LeftHand
    if hasattr(env, 'sys'):
        print(f"System loaded: {type(env.sys)}")
        
    # Check if there are XML files in the assets directory
    assets_dir = Path("assets")
    xml_files = list(assets_dir.glob("*.xml")) if assets_dir.exists() else []
    print(f"Available XML files: {[f.name for f in xml_files]}")
    
    # Try to load the model manually using the same pattern as HalfCheetah
    if xml_files:
        for xml_file in xml_files:
            print(f"\n🔄 Trying to load {xml_file.name}...")
            try:
                # Load the XML file directly
                sys_test = mjcf.load(str(xml_file))
                print(f"✅ Successfully loaded {xml_file.name}")
                print(f"System type: {type(sys_test)}")
                
                # Check if it has bodies
                if hasattr(sys_test, 'body'):
                    body = sys_test.body
                    if hasattr(body, 'name'):
                        body_names = body.name
                        if hasattr(body_names, 'shape'):
                            print(f"Number of bodies: {body_names.shape[0]}")
                            if body_names.shape[0] > 0:
                                print(f"Body names: {body_names[:min(5, len(body_names))]}")
                        else:
                            print(f"Body name: {body_names}")
                
                # Check if it has joints
                if hasattr(sys_test, 'joint'):
                    joint = sys_test.joint
                    if hasattr(joint, 'name'):
                        joint_names = joint.name
                        if hasattr(joint_names, 'shape'):
                            print(f"Number of joints: {joint_names.shape[0]}")
                            if joint_names.shape[0] > 0:
                                print(f"Joint names: {joint_names[:min(5, len(joint_names))]}")
                
                # Check if it has actuators
                if hasattr(sys_test, 'actuator'):
                    actuator = sys_test.actuator
                    if hasattr(actuator, 'name'):
                        actuator_names = actuator.name
                        if hasattr(actuator_names, 'shape'):
                            print(f"Number of actuators: {actuator_names.shape[0]}")
                            if actuator_names.shape[0] > 0:
                                print(f"Actuator names: {actuator_names[:min(5, len(actuator_names))]}")
                        else:
                            print(f"Actuator name: {actuator_names}")
                    else:
                        print("⚠️ No actuator names found")
                else:
                    print("⚠️ No actuators found in system")
                
                # If this system looks good, try to create a simple render
                if hasattr(sys_test, 'body') and hasattr(sys_test.body, 'name'):
                    print(f"\n🎨 Testing render with {xml_file.name}...")
                    try:
                        from brax.v1.physics.base import vec_to_arr
                        from brax.v1.physics.system import System
                        
                        # Create a simple QP state
                        if hasattr(sys_test, 'init_q'):
                            q_init = sys_test.init_q
                            qd_init = jnp.zeros_like(q_init)
                            
                            # Create pipeline state
                            if hasattr(env, 'pipeline_init'):
                                test_pipeline_state = env.pipeline_init(q_init, qd_init)
                                test_qp = test_pipeline_state.qp
                                
                                # Try to render
                                test_html = html.render(sys_test, [test_qp], height=600)
                                print("✅ Test render successful!")
                                HTML(test_html)
                                break
                            else:
                                print("⚠️ No pipeline_init method found")
                        else:
                            print("⚠️ No init_q found in system")
                        
                    except Exception as render_error:
                        print(f"❌ Test render failed: {render_error}")
                        continue
                        
            except Exception as e:
                print(f"❌ Failed to load {xml_file.name}: {e}")
                continue
    else:
        print("❌ No XML files found in assets directory")
        
except Exception as e:
    print(f"❌ Error during inspection: {e}")

print("\n💡 Solutions to try:")
print("1. Check if the XML file has proper visual geometries")
print("2. Verify that mesh files (.obj, .stl) exist and are referenced correctly")
print("3. Ensure actuators are defined in the XML")
print("4. Check if material definitions are present")
print("5. Verify the XML follows MuJoCo format correctly")

🔍 Inspecting LeftHand model loading...
LeftHand class: <class 'qdax.environments.left_hand.LeftHand'>
LeftHand MRO: (<class 'qdax.environments.left_hand.LeftHand'>, <class 'brax.v1.envs.env.Env'>, <class 'abc.ABC'>, <class 'object'>)
System loaded: <class 'brax.v1.physics.system.System'>
Available XML files: ['left_hand.xml', 'right_hand.xml', 'scene_left.xml', 'scene_right.xml']

🔄 Trying to load left_hand.xml...
❌ Failed to load left_hand.xml: [Errno 2] No such file or directory: 'assets\\assets\\palm_left.obj'

🔄 Trying to load right_hand.xml...
❌ Failed to load right_hand.xml: [Errno 2] No such file or directory: 'assets\\assets\\palm_right.obj'

🔄 Trying to load scene_left.xml...
❌ Failed to load scene_left.xml: [Errno 2] No such file or directory: 'assets\\assets\\palm_left.obj'

🔄 Trying to load scene_right.xml...
❌ Failed to load scene_right.xml: [Errno 2] No such file or directory: 'assets\\assets\\palm_right.obj'

💡 Solutions to try:
1. Check if the XML file has proper visual

In [34]:
# Try to recreate the environment with manual model loading
print("🔧 Attempting to recreate LeftHand environment with manual loading...")

try:
    # Import necessary classes
    from brax.envs.base import PipelineEnv
    from brax.io import mjcf
    from pathlib import Path
    
    # Find the best XML file to use
    assets_dir = Path("assets")
    xml_candidates = ["left_hand.xml", "scene_left.xml"]
    
    working_xml = None
    for xml_name in xml_candidates:
        xml_path = assets_dir / xml_name
        if xml_path.exists():
            try:
                print(f"🔄 Testing {xml_name}...")
                test_sys = mjcf.load(str(xml_path))
                
                # Check if it has the minimum required components
                has_bodies = hasattr(test_sys, 'body') and hasattr(test_sys.body, 'name')
                has_joints = hasattr(test_sys, 'joint') and hasattr(test_sys.joint, 'name')
                has_actuators = hasattr(test_sys, 'actuator')
                
                print(f"Bodies: {has_bodies}, Joints: {has_joints}, Actuators: {has_actuators}")
                
                if has_bodies and has_joints:
                    working_xml = xml_path
                    working_sys = test_sys
                    print(f"✅ {xml_name} looks promising!")
                    break
                    
            except Exception as e:
                print(f"❌ {xml_name} failed: {e}")
                continue
    
    if working_xml:
        print(f"\n🎨 Creating visualization with {working_xml.name}...")
        
        # Create a simple pipeline state
        try:
            # Get initial positions
            if hasattr(working_sys, 'init_q') and hasattr(working_sys, 'qd_size'):
                q_init = working_sys.init_q
                qd_init = jnp.zeros(working_sys.qd_size())
                
                print(f"Initial q shape: {q_init.shape}")
                print(f"Initial qd shape: {qd_init.shape}")
                
                # Try to create a pipeline state using the existing environment's method
                try:
                    # Use the original environment's pipeline_init
                    manual_pipeline_state = env.pipeline_init(q_init, qd_init)
                    manual_qp = manual_pipeline_state.qp
                    
                    print(f"Manual QP pos shape: {manual_qp.pos.shape}")
                    print(f"Manual QP rot shape: {manual_qp.rot.shape}")
                    
                    # Try to render with the working system
                    manual_html = html.render(working_sys, [manual_qp], height=700)
                    print("✅ Manual render created successfully!")
                    HTML(manual_html)
                    
                except Exception as pipeline_error:
                    print(f"❌ Pipeline creation failed: {pipeline_error}")
                    
                    # Try alternative approach - create QP manually
                    print("🔄 Trying alternative QP creation...")
                    try:
                        from brax.v1.physics.base import QP
                        
                        # Create a basic QP state
                        if hasattr(working_sys.body, 'name'):
                            body_names = working_sys.body.name
                            if hasattr(body_names, 'shape'):
                                num_bodies = body_names.shape[0]
                            else:
                                num_bodies = 1
                        else:
                            num_bodies = 1
                        
                        print(f"Creating QP for {num_bodies} bodies...")
                        
                        # Create basic positions and rotations
                        pos = jnp.zeros((num_bodies, 3))
                        rot = jnp.array([1.0, 0.0, 0.0, 0.0])
                        rot = jnp.tile(rot, (num_bodies, 1))
                        
                        vel = jnp.zeros((num_bodies, 3))
                        ang = jnp.zeros((num_bodies, 3))
                        
                        manual_qp = QP(pos=pos, rot=rot, vel=vel, ang=ang)
                        
                        # Try to render
                        alt_html = html.render(working_sys, [manual_qp], height=700)
                        print("✅ Alternative render created!")
                        HTML(alt_html)
                        
                    except Exception as alt_error:
                        print(f"❌ Alternative approach failed: {alt_error}")
                        print("The XML might be missing essential visual elements")
                        
            else:
                print("❌ System missing init_q or qd_size")
                
        except Exception as create_error:
            print(f"❌ Error creating pipeline state: {create_error}")
    else:
        print("❌ No suitable XML file found")
        print("The XML files might not be properly formatted for MuJoCo/Brax")
        
except Exception as e:
    print(f"❌ Critical error in manual loading: {e}")
    print("Check if the XML files follow the MuJoCo format correctly")

🔧 Attempting to recreate LeftHand environment with manual loading...
🔄 Testing left_hand.xml...
❌ left_hand.xml failed: [Errno 2] No such file or directory: 'assets\\assets\\palm_left.obj'
🔄 Testing scene_left.xml...
❌ scene_left.xml failed: [Errno 2] No such file or directory: 'assets\\assets\\palm_left.obj'
❌ No suitable XML file found
The XML files might not be properly formatted for MuJoCo/Brax


# Left Hand Visualization

The render above shows the LEAP Hand (Left Hand) in action. The visualization includes:

- **Hand Model**: A detailed 3D model of the left hand with all joints and links
- **Animation**: Shows the hand performing random movements over several time steps
- **Physics**: Realistic physics simulation using the Brax physics engine

You can interact with the 3D visualization by:
- Rotating the view by clicking and dragging
- Zooming in/out with the mouse wheel
- Playing/pausing the animation with the controls

The hand model is based on the LEAP Hand robot design and includes all the necessary degrees of freedom for complex manipulation tasks.

In [ ]:
# Test the updated LeftHand implementation
print("🔄 Testing updated LeftHand implementation...")

try:
    # Reload the module to get the updated version
    import importlib
    import qdax.environments.left_hand
    importlib.reload(qdax.environments.left_hand)
    from qdax.environments.left_hand import LeftHand
    
    print("✅ Successfully reloaded LeftHand module")
    
    # Create new environment with updated implementation
    print("🔄 Creating new LeftHand environment...")
    new_env = LeftHand(backend='generalized')
    print("✅ New LeftHand environment created successfully!")
    
    # Test basic functionality
    print("🔄 Testing environment reset...")
    key = jax.random.PRNGKey(42)
    new_state = new_env.reset(key)
    print("✅ Environment reset successful!")
    
    print(f"Action size: {new_env.action_size}")
    print(f"Observation size: {new_env.observation_size}")
    
    # Generate a simple trajectory
    print("🔄 Generating trajectory...")
    new_states = [new_state]
    current_state = new_state
    
    for i in range(10):
        action = jax.random.normal(key, shape=(new_env.action_size,)) * 0.2
        key, subkey = jax.random.split(key)
        current_state = new_env.step(current_state, action)
        new_states.append(current_state)
    
    print(f"✅ Generated {len(new_states)} states")
    
    # Extract QPs for rendering
    print("🔄 Extracting QPs for rendering...")
    new_qps = []
    for state in new_states:
        if hasattr(state, 'pipeline_state') and hasattr(state.pipeline_state, 'qp'):
            new_qps.append(state.pipeline_state.qp)
        else:
            print(f"⚠️ Could not extract QP from state")
            break
    
    print(f"✅ Extracted {len(new_qps)} QPs")
    
    if new_qps:
        # Check QP properties
        first_qp = new_qps[0]
        print(f"QP pos shape: {first_qp.pos.shape}")
        print(f"QP rot shape: {first_qp.rot.shape}")
        print(f"Position range: {jnp.min(first_qp.pos):.3f} to {jnp.max(first_qp.pos):.3f}")
        
        # Try to render
        print("🎨 Attempting to render with new implementation...")
        try:
            new_html = html.render(new_env.sys, new_qps, height=700)
            print("✅ Render successful with new implementation!")
            HTML(new_html)
        except Exception as render_error:
            print(f"❌ Render failed: {render_error}")
            
            # Debug system properties
            print("🔍 Debugging new system:")
            print(f"System type: {type(new_env.sys)}")
            if hasattr(new_env.sys, 'body'):
                body = new_env.sys.body
                if hasattr(body, 'name'):
                    body_names = body.name
                    if hasattr(body_names, 'shape'):
                        print(f"Number of bodies: {body_names.shape[0]}")
                        if body_names.shape[0] > 0:
                            print(f"Body names: {body_names[:min(5, len(body_names))]}")
    else:
        print("❌ No QPs available for rendering")
        
except Exception as e:
    print(f"❌ Error testing new implementation: {e}")
    import traceback
    traceback.print_exc()
    print("Check if the XML file is properly formatted for MuJoCo")

In [ ]:
# Diagnose gray render issue and create working XML
print("🔍 Diagnosing gray render issue...")

# First, let's check what's actually in the XML files
from pathlib import Path
import xml.etree.ElementTree as ET

assets_dir = Path("assets")
if assets_dir.exists():
    xml_files = list(assets_dir.glob("*.xml"))
    for xml_file in xml_files:
        print(f"\n📄 Analyzing {xml_file.name}:")
        try:
            with open(xml_file, 'r') as f:
                content = f.read()
            
            # Parse XML to check structure
            root = ET.fromstring(content)
            
            # Count important elements
            bodies = root.findall('.//body')
            geoms = root.findall('.//geom')
            materials = root.findall('.//material')
            meshes = root.findall('.//mesh')
            actuators = root.findall('.//actuator')
            
            print(f"  Bodies: {len(bodies)}")
            print(f"  Geoms: {len(geoms)}")
            print(f"  Materials: {len(materials)}")
            print(f"  Meshes: {len(meshes)}")
            print(f"  Actuators: {len(actuators)}")
            
            # Check if geoms have visual properties
            visual_geoms = 0
            for geom in geoms:
                if geom.get('type') and geom.get('size'):
                    visual_geoms += 1
                elif geom.get('mesh'):
                    visual_geoms += 1
            print(f"  Visual geoms: {visual_geoms}")
            
            # Check for common issues
            if len(geoms) == 0:
                print("  ⚠️ No geometries found - this will cause gray render!")
            if len(actuators) == 0:
                print("  ⚠️ No actuators found - hand won't move!")
            if len(materials) == 0:
                print("  ⚠️ No materials found - may cause rendering issues!")
                
        except Exception as e:
            print(f"  ❌ Error parsing XML: {e}")

# Create a minimal working hand XML if needed
print("\n🔧 Creating minimal working hand XML...")

minimal_hand_xml = '''<?xml version="1.0" ?>
<mujoco model="left_hand_minimal">
  <compiler angle="radian" meshdir="." texturedir="."/>
  
  <option gravity="0 0 -9.81" timestep="0.002"/>
  
  <default>
    <joint damping="0.5" stiffness="10"/>
    <geom friction="1 0.1 0.1" rgba="0.7 0.5 0.3 1" type="capsule"/>
    <motor ctrlrange="-1 1" ctrllimited="true"/>
  </default>
  
  <asset>
    <material name="hand_material" rgba="0.8 0.6 0.4 1"/>
    <material name="fingertip_material" rgba="0.9 0.7 0.5 1"/>
  </asset>
  
  <worldbody>
    <light directional="true" ambient="0.2 0.2 0.2" diffuse="0.8 0.8 0.8" specular="0.3 0.3 0.3" castshadow="false" pos="0 0 4" dir="0 0 -1"/>
    <geom name="floor" pos="0 0 -0.1" size="2 2 0.05" type="box" rgba="0.5 0.5 0.5 1"/>
    
    <body name="palm" pos="0 0 0.1">
      <geom name="palm_geom" type="box" size="0.05 0.08 0.02" material="hand_material"/>
      <inertial pos="0 0 0" mass="0.5" diaginertia="0.01 0.01 0.01"/>
      
      <!-- Index finger -->
      <body name="index_proximal" pos="0.04 0.06 0.02">
        <joint name="index_mcp" type="hinge" axis="1 0 0" range="-0.5 1.5"/>
        <geom name="index_prox_geom" type="capsule" fromto="0 0 0 0 0 0.03" size="0.008" material="hand_material"/>
        <inertial pos="0 0 0.015" mass="0.05" diaginertia="0.001 0.001 0.001"/>
        
        <body name="index_distal" pos="0 0 0.03">
          <joint name="index_pip" type="hinge" axis="1 0 0" range="-0.2 1.2"/>
          <geom name="index_dist_geom" type="capsule" fromto="0 0 0 0 0 0.025" size="0.006" material="hand_material"/>
          <geom name="index_tip_geom" pos="0 0 0.025" type="sphere" size="0.008" material="fingertip_material"/>
          <inertial pos="0 0 0.0125" mass="0.03" diaginertia="0.0005 0.0005 0.0005"/>
        </body>
      </body>
      
      <!-- Middle finger -->
      <body name="middle_proximal" pos="0.02 0.08 0.02">
        <joint name="middle_mcp" type="hinge" axis="1 0 0" range="-0.5 1.5"/>
        <geom name="middle_prox_geom" type="capsule" fromto="0 0 0 0 0 0.035" size="0.008" material="hand_material"/>
        <inertial pos="0 0 0.0175" mass="0.05" diaginertia="0.001 0.001 0.001"/>
        
        <body name="middle_distal" pos="0 0 0.035">
          <joint name="middle_pip" type="hinge" axis="1 0 0" range="-0.2 1.2"/>
          <geom name="middle_dist_geom" type="capsule" fromto="0 0 0 0 0 0.03" size="0.006" material="hand_material"/>
          <geom name="middle_tip_geom" pos="0 0 0.03" type="sphere" size="0.008" material="fingertip_material"/>
          <inertial pos="0 0 0.015" mass="0.03" diaginertia="0.0005 0.0005 0.0005"/>
        </body>
      </body>
      
      <!-- Ring finger -->
      <body name="ring_proximal" pos="-0.02 0.08 0.02">
        <joint name="ring_mcp" type="hinge" axis="1 0 0" range="-0.5 1.5"/>
        <geom name="ring_prox_geom" type="capsule" fromto="0 0 0 0 0 0.032" size="0.007" material="hand_material"/>
        <inertial pos="0 0 0.016" mass="0.04" diaginertia="0.001 0.001 0.001"/>
        
        <body name="ring_distal" pos="0 0 0.032">
          <joint name="ring_pip" type="hinge" axis="1 0 0" range="-0.2 1.2"/>
          <geom name="ring_dist_geom" type="capsule" fromto="0 0 0 0 0 0.025" size="0.005" material="hand_material"/>
          <geom name="ring_tip_geom" pos="0 0 0.025" type="sphere" size="0.007" material="fingertip_material"/>
          <inertial pos="0 0 0.0125" mass="0.03" diaginertia="0.0005 0.0005 0.0005"/>
        </body>
      </body>
      
      <!-- Thumb -->
      <body name="thumb_proximal" pos="0.05 0.02 0.01" euler="0 0 1.2">
        <joint name="thumb_cmc" type="hinge" axis="0 1 0" range="-0.8 0.8"/>
        <geom name="thumb_prox_geom" type="capsule" fromto="0 0 0 0 0 0.025" size="0.009" material="hand_material"/>
        <inertial pos="0 0 0.0125" mass="0.06" diaginertia="0.001 0.001 0.001"/>
        
        <body name="thumb_distal" pos="0 0 0.025">
          <joint name="thumb_mcp" type="hinge" axis="1 0 0" range="-0.3 1.0"/>
          <geom name="thumb_dist_geom" type="capsule" fromto="0 0 0 0 0 0.02" size="0.007" material="hand_material"/>
          <geom name="thumb_tip_geom" pos="0 0 0.02" type="sphere" size="0.009" material="fingertip_material"/>
          <inertial pos="0 0 0.01" mass="0.04" diaginertia="0.0005 0.0005 0.0005"/>
        </body>
      </body>
    </body>
  </worldbody>
  
  <actuator>
    <motor name="index_mcp_motor" joint="index_mcp" gear="20"/>
    <motor name="index_pip_motor" joint="index_pip" gear="15"/>
    <motor name="middle_mcp_motor" joint="middle_mcp" gear="20"/>
    <motor name="middle_pip_motor" joint="middle_pip" gear="15"/>
    <motor name="ring_mcp_motor" joint="ring_mcp" gear="18"/>
    <motor name="ring_pip_motor" joint="ring_pip" gear="13"/>
    <motor name="thumb_cmc_motor" joint="thumb_cmc" gear="25"/>
    <motor name="thumb_mcp_motor" joint="thumb_mcp" gear="20"/>
  </actuator>
</mujoco>'''

# Save the minimal XML
minimal_xml_path = Path("assets") / "minimal_hand.xml"
minimal_xml_path.parent.mkdir(exist_ok=True)

with open(minimal_xml_path, 'w') as f:
    f.write(minimal_hand_xml)

print(f"✅ Created minimal hand XML at: {minimal_xml_path}")
print("This XML includes:")
print("- Visible geometries (capsules and spheres)")
print("- Materials and colors")
print("- Proper lighting")
print("- Floor for reference")
print("- 8 actuators for hand movement")
print("- Proper inertial properties")

In [ ]:
# Test render with minimal working XML
print("🎨 Testing render with minimal hand XML...")

try:
    from brax.io import mjcf
    from brax.v1.io import html
    from pathlib import Path
    
    # Load the minimal XML directly
    minimal_xml_path = Path("assets") / "minimal_hand.xml"
    
    if minimal_xml_path.exists():
        print(f"✅ Loading minimal XML from: {minimal_xml_path}")
        
        # Load system with mjcf
        minimal_sys = mjcf.load(str(minimal_xml_path))
        print("✅ Minimal system loaded successfully!")
        
        # Check system properties
        print(f"System type: {type(minimal_sys)}")
        if hasattr(minimal_sys, 'body'):
            body = minimal_sys.body
            if hasattr(body, 'name'):
                body_names = body.name
                if hasattr(body_names, 'shape'):
                    print(f"Number of bodies: {body_names.shape[0]}")
                    if body_names.shape[0] > 0:
                        print(f"Body names: {body_names[:min(8, len(body_names))]}")
        
        if hasattr(minimal_sys, 'actuator'):
            actuator = minimal_sys.actuator
            if hasattr(actuator, 'name'):
                actuator_names = actuator.name
                if hasattr(actuator_names, 'shape'):
                    print(f"Number of actuators: {actuator_names.shape[0]}")
                    if actuator_names.shape[0] > 0:
                        print(f"Actuator names: {actuator_names[:min(8, len(actuator_names))]}")
        
        # Create a simple QP state for rendering
        print("🔄 Creating QP state...")
        if hasattr(minimal_sys, 'init_q'):
            q_init = minimal_sys.init_q
            print(f"Initial q shape: {q_init.shape}")
            print(f"Initial q values: {q_init}")
            
            # Create simple pipeline environment for QP generation
            from brax.envs.base import PipelineEnv
            
            class MinimalHandEnv(PipelineEnv):
                def __init__(self):
                    super().__init__(sys=minimal_sys, backend='generalized')
            
            minimal_env = MinimalHandEnv()
            
            # Create initial state
            qd_init = jnp.zeros(minimal_sys.qd_size())
            pipeline_state = minimal_env.pipeline_init(q_init, qd_init)
            qp = pipeline_state.qp
            
            print(f"QP pos shape: {qp.pos.shape}")
            print(f"QP rot shape: {qp.rot.shape}")
            print(f"Position range: {jnp.min(qp.pos):.3f} to {jnp.max(qp.pos):.3f}")
            
            # Create animated sequence
            print("🔄 Creating animated sequence...")
            qps = [qp]
            
            # Generate some finger movements
            for i in range(15):
                # Create hand movements - gradually close fingers
                action = jnp.array([
                    0.3 * jnp.sin(i * 0.3),      # index_mcp
                    0.4 * jnp.sin(i * 0.3 + 1),  # index_pip
                    0.3 * jnp.sin(i * 0.3 + 0.5), # middle_mcp
                    0.4 * jnp.sin(i * 0.3 + 1.5), # middle_pip
                    0.2 * jnp.sin(i * 0.3 + 2),   # ring_mcp
                    0.3 * jnp.sin(i * 0.3 + 2.5), # ring_pip
                    0.2 * jnp.sin(i * 0.2),       # thumb_cmc
                    0.3 * jnp.sin(i * 0.2 + 1),   # thumb_mcp
                ])
                
                pipeline_state = minimal_env.pipeline_step(pipeline_state, action)
                qps.append(pipeline_state.qp)
            
            print(f"✅ Created {len(qps)} animation frames")
            
            # Render the animation
            print("🎨 Rendering hand animation...")
            try:
                hand_html = html.render(minimal_sys, qps, height=800)
                print("✅ SUCCESS! Hand render created with visible geometry!")
                HTML(hand_html)
            except Exception as render_error:
                print(f"❌ Render failed: {render_error}")
                
                # Try single frame
                print("🔄 Trying single frame render...")
                try:
                    single_html = html.render(minimal_sys, [qps[0]], height=800)
                    print("✅ Single frame render successful!")
                    HTML(single_html)
                except Exception as single_error:
                    print(f"❌ Single frame render failed: {single_error}")
        else:
            print("❌ No init_q found in minimal system")
    else:
        print("❌ Minimal XML file not found")
        
except Exception as e:
    print(f"❌ Error testing minimal XML: {e}")
    import traceback
    traceback.print_exc()

print("\n💡 If this works, you should see:")
print("- A brown/orange colored hand")
print("- Finger movements (opening/closing)")
print("- A gray floor for reference")
print("- Proper 3D perspective and lighting")